In [1]:
import pandas as pd
import cufflinks as cf
import warnings
import sys
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chisquare
from  functools import reduce
import re
from plotly.offline import plot,iplot
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.express as px#graficos express
import sys
warnings.filterwarnings("ignore")
cf.go_offline()
pd.set_option("display.max_columns",200)

In [2]:
def chi_square(df,col,valor_miss):
    x_i=df[col].fillna(valor_miss).value_counts()
    k=x_i.sum()
    p_i=df[col].dropna().value_counts(1)
    m_i=k*p_i
    chi=chisquare(f_obs=x_i,f_exp=m_i)
    p_val=chi.pvalue
    alpha=0.05
    if p_val<alpha:
        print("Rechazamos HO(La porporción de categorias es la misma que la general)")
    else:
        print("Aceptamos HO(La porporción de categorias es la misma que la general)")

In [3]:
def other(df,col,n):
    aux=list(df[col].value_counts(1).index[n:].values)
    dictio_aux=dict(zip(aux,["otro"]*len(aux)))
    df[col]=df[col].replace(dictio_aux)
    return df

def multiple_replace(string, rep_dict):
    pattern = re.compile("|".join([re.escape(k) for k in sorted(rep_dict,key=len,reverse=True)]), flags=re.DOTALL)
    return pattern.sub(lambda x: rep_dict[x.group(0)]+"_", string)

In [4]:
## Multicolinealidad Categoricas##
def multicolinealidad_cate(df,y,threshold=5,only_final_vif=True):
    '''
    Función para calcular el VIF Genralizado, (GVIF, Fox and Monette 1992)
    Las variables no deben de deben de estar transformadas con OneHotEncoder ya que la función lo realiza internamnete.
    Args:
        df
        y
        threshold
        only_final_vif
    
    Returns:
        pandas data frame: a data frame, indexed by factor of the GVIF, GVIF^(1/2Df), VIF^(1/2Df)^2 
        dictionary: Dictionary of column names (keys) and GVIF ** (1 / (2 * Df)) ** 2 (values)
    '''

    df_x = df.drop(y,axis=1)
    #Guardamos los nombres de las columnas añadiendo el prefijo

    onehot_list = list(df_x.select_dtypes(include=['category', 'object', 'string']).columns)
    #Generamos las variables dummy 
    
    df_1hot = pd.get_dummies(df_x, drop_first=True, dummy_na=False, prefix_sep='_')


    #Dataframe vacio para guardar los resultados
    gvif_df = pd.DataFrame(columns = ['factor','GVIF','Df','GVIF^(1/2Df)', 'VIF'])

    # Iteramos sobre las columas
    for (columnName, columnData) in df_x.iteritems():
        #Si se generaron las de una columna, es decir la variable tiene más de dos posibilidades utilizamos todas las variables creadas
        
        if columnName in onehot_list:
            X1 = df_1hot.loc[:, df_1hot.columns.str.startswith(columnName)]
            X2 = df_1hot.loc[:, ~df_1hot.columns.str.startswith(columnName)]
        else:
            X1 = df_1hot[[columnName]].values
            X2 = df_1hot.loc[:, df_1hot.columns != columnName].values
        display(X1)
        # Calculamose gvif
        #A la matriz de correlación de las variables codificadas en caliente del atributo en consideración.
        #B la matriz de correlación de todos los demás atributos del conjunto de datos (uno codificado en dummy y numérico) excluyendo los de A.
        #C la matriz de correlación de las variables consideradas tanto en A como en B.
        #GVIF= (det(A)*det(B))/det(C)
        display(np.corrcoef(X1, rowvar=False))
        gvif = np.linalg.det(np.array(np.corrcoef(X1, rowvar=False), ndmin=2)) * np.linalg.det(np.corrcoef(X2, rowvar=False)) / np.linalg.det(np.corrcoef(np.append(X1, X2, axis=1), rowvar=False))
        #(GVIF) elevado a (1 / (2 * grados de libertad))
        #Los grados de libertad es 1-n (n=Número de opciones que tiene la variable categorica)
        gvif_12df = np.power(gvif, 1 / (2 * X1.shape[1]))
        gvif_12df_sq = gvif_12df ** 2
        DF =  X1.shape[1]
    

        # 
        new_row = {'factor': columnName, 'GVIF': gvif,'Df':DF, 'GVIF^(1/2Df)': gvif_12df, 'VIF': gvif_12df_sq}
        gvif_df = gvif_df.append(new_row, ignore_index=True)

    gvif_df = gvif_df.set_index('factor')
    if only_final_vif:
        gvif_df_final = gvif_df.drop(['GVIF','Df','GVIF^(1/2Df)'],axis=1)
    else:
        gvif_df_final = gvif_df 
    gvif_filter = gvif_df.loc[gvif_df['VIF'] >= threshold]['VIF'].to_dict()
    if gvif_filter:
        for i in gvif_filter.keys():
            df_x_m = df_x.drop([i],axis=1)
    else:
        df_x_m = df_x
    df_m=pd.concat([df_x_m,df[y]],axis=1)
    return gvif_df_final,gvif_filter,df_m


In [5]:
from libreria import rename_cols,completitud,text_clean,box,histogram

In [6]:
df=pd.read_csv("../Data/visualizaciones/dataset_house_prices.csv")

## Variables

**El problema**

En análisis exploratorio tiene el objetivo de entender el significado y la relevancia de cada variable con respecto al problema. Además apartir de este análisis se pueden determinar que acciones tomar para cada variable En este caso:

- La relevancia de la variable en la compra de una casa.
- La importancia de la variable.
- Solapamiento con otras variables.


Variable objetivo : 'SalePrice'.

- MSSubClass: clase de construcción
- MSZoning: clasificación de la zona
- LotFrontage: pies lineales de calle de la parcela
- LotArea: tamaño de la parcela en pies cuadrados
- Street: tipo de acceso por carretera
- Alley: tipo de acceso al callejón
- LotShape: forma de la parcela
- LandContour: planitud de la parcela
- Utilities: servicios públicos disponibles
- LotConfig: Configuración de parcela
- LandSlope: pendiente de la parcela
- Neighborhood: ubicación física dentro de los límites de la ciudad de Ames
- Condition1: proximidad a la carretera principal o al ferrocarril
- Condition2: proximidad a la carretera principal o al ferrocarril (si hay un segundo)
- BldgType: tipo de vivienda
- HouseStyle: estilo de vivienda
- OverallQual: calidad general del material y del acabado
- OverallCond: condición general
- YearBuilt: fecha original de construcción
- YearRemodAdd: fecha de remodelación
- RoofStyle: tipo de cubierta
- RoofMatl: material del techo
- Exterior1st: revestimiento exterior de la casa
- Exterior2nd: revestimiento exterior de la casa (si hay más de un material)
- MasVnrType: tipo de revestimiento de mampostería
- MasVnrArea: área de revestimiento de mampostería en pies cuadrados
- ExterQual: calidad del material exterior
- ExterCond: estado del material en el exterior
- Foundation: tipo de cimentación
- BsmtQual: altura del sótano
- BsmtCond: estado general del sótano
- BsmtExposure: paredes del sótano a nivel de calle o de jardín
- BsmtFinType1: calidad del área acabada del sótano
- BsmtFinSF1: pies cuadrados de la superficie acabada tipo 1
- BsmtFinType2: calidad de la segunda superficie acabada (si existe)
- BsmtFinSF2: Pies cuadrados de la superficie acabada tipo 2
- BsmtUnfSF: pies cuadrados del área sin terminar del sótano
- TotalBsmtSF: pies cuadrados totales del sótano
- Heating: tipo de calefacción
- HeatingQC: calidad y estado de la calefacción
- CentralAir: aire acondicionado central
- Electrical: sistema eléctrico
- 1erFlrSF: área en pies cuadrados de la primera planta (o planta baja)
- 2ndFlrSF: área en pies cuadrados de la segunda planta
- LowQualFinSF: pies cuadrados acabados de baja calidad (todos los pisos)
- GrLivArea: superficie habitable por encima del nivel del suelo en pies cuadrados
- BsmtFullBath: cuartos de baño completos en el sótano
- BsmtHalfBath: medio baño del sótano
- FullBath: baños completos sobre el nivel del suelo
- HalfBath: medios baños sobre el nivel del suelo
- Bedroom: número de dormitorios por encima del nivel del sótano
- Kitchen: número de cocinas
- KitchenQual: calidad de la cocina
- TotRmsAbvGrd: total de habitaciones por encima del nivel del suelo (no incluye baños)
- Functional: valoración de la funcionalidad de la vivienda
- Fireplaces: número de chimeneas
- FireplaceQu: calidad de la chimenea
- GarageType: ubicación del garaje
- GarageYrBlt: año de construcción del garaje
- GarageFinish: acabado interior del garaje
- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados
- GarageQual: calidad de garaje
- GarageCond: condición de garaje
- PavedDrive: calzada asfaltada
- WoodDeckSF: area de plataforma de madera en pies cuadrados
- OpenPorchSF: área de porche abierto en pies cuadrados
- EnclosedPorch: área de porche cerrada en pies cuadrados
- 3SsnPorch: área de porche de tres estaciones en pies cuadrados
- ScreenPorch: superficie acristalada del porche en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados
- PoolQC: calidad de la piscina
- Fence: calidad de la valla
- MiscFeature: característica miscelánea no cubierta en otras categorías
- MiscVal: valor en dólares de la característica miscelánea
- MoSold: mes de venta
- YrSold: año de venta
- SaleType: tipo de venta
- SaleCondition: Condiciones de venta

In [7]:
v_feats=["MSSubClass","MSZoning","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope","Condition1","Condition2","BldgType","HouseStyle","OverallQual","OverallCond",
        "YearBuilt","YearRemodAdd","RoofStyle","RoofMatl","MasVnrType","ExterQual",
        "ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
        "Heating","HeatingQC","CentralAir","Electrical","LowQualFinSF","BsmtHalfBath","FullBath","KitchenAbvGr","KitchenQual","Functional","Fireplaces","FireplaceQu",
        "GarageType","GarageYrBlt","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence",
        "MiscFeature","MoSold","YrSold","SaleType","SaleCondition"]
c_feats=["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","1stFlrSF",
        "2ndFlrSF","GrLivArea","GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch",
        "PoolArea","MiscVal","BedroomAbvGr","HalfBath","GarageCars","BsmtFullBath","TotRmsAbvGrd"]
tgt=["SalePrice"]

df=rename_cols(df,v_feats,"v_")
df=rename_cols(df,c_feats,"c_")
df=rename_cols(df,tgt,"tgt_")

In [8]:
#Eliminamos aquellas variables que que hacen referencia a la venta
df.drop(columns=["v_SaleType","v_SaleCondition"],inplace=True)

In [9]:
df.drop(columns=["Neighborhood","Exterior2nd","Exterior1st","v_YrSold","v_MoSold"],inplace=True)

In [10]:
#NO hay duplicados
df[df.duplicated(subset=["Id"])]

,Id,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_Alley,v_LotShape,v_LandContour,v_Utilities,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,c_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_FireplaceQu,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,v_PoolQC,v_Fence,v_MiscFeature,c_MiscVal,tgt_SalePrice


In [11]:
#Variables para limpiar el texto
features=list(filter(lambda x: x not in ["v_GarageYrBlt","v_YearBuilt"],df.filter(like="v_") ))


for col in features:
    df[col]=df[col].fillna("nan").astype(str).map(text_clean).replace("nan",np.nan)

In [12]:
histogram(df,"tgt_SalePrice",5,"tgt_SalePrice").show()

In [13]:
miss=completitud(df)

In [14]:
miss

,columna,total,completitud
0,v_PoolQC,1453,0.479452
1,v_MiscFeature,1406,3.698630
2,v_Alley,1369,6.232877
3,v_Fence,1179,19.246575
4,v_FireplaceQu,690,52.739726
...,...,...,...
69,c_MiscVal,0,100.000000
70,v_HeatingQC,0,100.000000
71,v_CentralAir,0,100.000000
72,c_2ndFlrSF,0,100.000000


In [15]:
null_cols=list(miss[miss["completitud"]<80]["columna"].values)

In [16]:
null_cols

['v_PoolQC', 'v_MiscFeature', 'v_Alley', 'v_Fence', 'v_FireplaceQu']

In [17]:
df=df.drop(columns=null_cols)

In [18]:
df["v_MSSubClass"]=df["v_MSSubClass"].astype(int)

In [19]:
df_multi=df.copy()

In [20]:
df_multi=df.copy()

## Isolation Forest

**La clase sklearn.ensemble.IsolationForest incorpora las principales funcionalidades que se necesitan a la hora de trabajar con modelos Isolation Forest. Los principales argumentos para entrenar este tipo de modelos son:**

- n_estimators: número de árboles que forman el modelo.

- max_samples: número de observaciones empleadas para entrenar cada árbol.

- max_features : El número de características que se extraerán de X para entrenar cada estimador base.

- contamination: proporción de anomalías esperadas en los datos de entrenamiento. En base a este valor, se establece el límite acorde al cual se clasifican las observaciones en normales o anómalas.

- random_state: semilla para garantizar la reproducibilidad de los resultados.

- Se procede a entrenar un modelo asumiendo que hay un 1% de observaciones anómalas en el conjunto de entrenamiento.

In [21]:
from sklearn.ensemble import IsolationForest

In [22]:
df=df.drop(columns=["Id"])

In [23]:
df.shape

(1460, 68)

In [24]:
df_iso=df.filter(like="c_").dropna()

In [25]:
#max_features=1
max_features=df_iso.shape[1]
n_estimators=50
max_samples='auto'
contamination=float(0.2)
model=IsolationForest(max_features = max_features, n_estimators=n_estimators, max_samples=max_samples, contamination=contamination)
model.fit(df_iso)

IsolationForest(contamination=0.2, max_features=23, n_estimators=50)

- Los modelos IsolationForest tienen dos métodos de predicción con los que se obtiene distinta información.

- Con el método predict() se devuelve directamente la clasificación de anomalía (-1) o no anomalía (1) acorde a la proporción de contaminación que se ha indicado en la definición del modelo.

In [26]:
df_iso["outlier"]=model.predict(df_iso)

In [27]:
df_iso["outlier"].value_counts()

 1    956
-1    239
Name: outlier, dtype: int64

In [28]:
df_iso[df_iso["outlier"]==-1].shape[0]/df_iso.shape[0]

0.2

In [29]:
indices=list(df_iso[df_iso["outlier"]==1].index)

In [30]:
df=df.iloc[indices]

In [31]:
df.reset_index(drop=True,inplace=True)

## MISSINGS

### Variables unitarias

In [32]:
#Eliminamos Variables unarias

In [33]:
df[["v_Utilities","v_Street"]].describe()

,v_Utilities,v_Street
count,956,956
unique,1,2
top,allpub,pave
freq,956,954


In [34]:
df.drop(columns=["v_Utilities"],inplace=True)

In [35]:
miss=completitud(df)

miss_cols=list(miss[miss["completitud"]<100]["columna"].values)

In [36]:
miss

,columna,total,completitud
0,v_GarageQual,41,95.711297
1,v_GarageType,41,95.711297
2,v_GarageYrBlt,41,95.711297
3,v_GarageFinish,41,95.711297
4,v_GarageCond,41,95.711297
...,...,...,...
62,c_BsmtUnfSF,0,100.000000
63,c_TotalBsmtSF,0,100.000000
64,c_MiscVal,0,100.000000
65,v_YearBuilt,0,100.000000


In [37]:
miss_cols

['v_GarageQual',
 'v_GarageType',
 'v_GarageYrBlt',
 'v_GarageFinish',
 'v_GarageCond',
 'v_BsmtExposure',
 'v_BsmtFinType2',
 'v_BsmtQual',
 'v_BsmtFinType1',
 'v_BsmtCond',
 'v_Electrical']

### NORMALIZAMOS DISCRETAS

In [38]:
v_=list(filter(lambda x:"v_" in x,miss_cols))
v_=[x for x in v_ if x!="v_GarageYrBlt"]

In [39]:
for col in v_:
    print(col)
    print(df[col].isnull().sum())
    df[col]=df[col].fillna("nan").astype(str).map(text_clean).replace("nan",np.nan)
    print(df[col].isnull().sum())

v_GarageQual
41
41
v_GarageType
41
41
v_GarageFinish
41
41
v_GarageCond
41
41
v_BsmtExposure
23
23
v_BsmtFinType2
22
22
v_BsmtQual
22
22
v_BsmtFinType1
22
22
v_BsmtCond
22
22
v_Electrical
1
1


In [40]:
for col in v_:
    print(col)
    display(df[col].value_counts(1))
    print("\n")

v_GarageQual


ta    0.937705
fa    0.045902
gd    0.010929
po    0.003279
ex    0.002186
Name: v_GarageQual, dtype: float64



v_GarageType


attchd     0.603279
detchd     0.328962
builtin    0.044809
basment    0.014208
carport    0.005464
2types     0.003279
Name: v_GarageType, dtype: float64



v_GarageFinish


...

unf    0.339400
glq    0.263383
alq    0.156317
rec    0.099572
blq    0.093148
lwq    0.048180
Name: v_BsmtFinType1, dtype: float64



v_BsmtCond


ta    0.919700
gd    0.044968
fa    0.034261
po    0.001071
Name: v_BsmtCond, dtype: float64



v_Electrical


sbrkr    0.899476
fusea    0.074346
fusef    0.021990
fusep    0.003141
mix      0.001047
Name: v_Electrical, dtype: float64

In [41]:
##"v_GarageType"
##"v_Electrical"
#MasVnrType

In [42]:
dictionary={"v_BldgType":1,"v_Foundation":2,"v_Functional":1,"v_Heating":1,"v_HouseStyle":3,"v_LandContour":1,
"v_LandSlope":1,"v_LotConfig":2,"v_LotShape":1,"v_MSZoning":2,"v_RoofMatl":1,"v_RoofStyle":2,"v_Condition2":1,"v_Condition1":1,"v_BsmtFinType2":1,"v_BsmtFinType1":3,"v_BsmtExposure":1}

In [43]:
for i,j in dictionary.items():
    print(i)
    df=other(df,i,j)
    print(df[i].value_counts())

v_BldgType
1fam    803
otro    153
Name: v_BldgType, dtype: int64
v_Foundation
pconc     420
cblock    400
otro      136
Name: v_Foundation, dtype: int64
v_Functional
typ     893
otro     63
Name: v_Functional, dtype: int64
v_Heating
gasa    934
otro     22
Name: v_Heating, dtype: int64
v_HouseStyle
1story    482
2story    278
1 5fin    115
otro       81
Name: v_HouseStyle, dtype: int64
v_LandContour
lvl     869
otro     87
Name: v_LandContour, dtype: int64
v_LandSlope
gtl     920
otro     36
Name: v_LandSlope, dtype: int64
v_LotConfig
inside    744
corner    152
otro       60
Name: v_LotConfig, dtype: int64
v_LotShape
reg     703
otro    253
Name: v_LotShape, dtype: int64
v_MSZoning
rl      729
rm      161
otro     66
Name: v_MSZoning, dtype: int64
v_RoofMatl
compshg    949
otro         7
Name: v_RoofMatl, dtype: int64
v_RoofStyle
gable    776
hip      164
otro      16
Name: v_RoofStyle, dtype: int64
v_Condition2
norm    947
otro      9
Name: v_Condition2, dtype: int64
v_Condition1
no

In [44]:
dictio_GarageType=dict(zip(list(df["v_GarageType"].value_counts().index[-3:]),["otro"]*3))
dictio_electrical=dict(zip(list(df["v_Electrical"].value_counts().index[-3:]),["otro"]*3))
dictio_MasVnrType=dict(zip(list(df["v_MasVnrType"].value_counts().index[-2:]),["otro"]*2))

In [45]:
df["v_GarageType"]=df["v_GarageType"].replace(dictio_GarageType)
df["v_Electrical"]=df["v_Electrical"].replace(dictio_electrical)
df["v_MasVnrType"]=df["v_MasVnrType"].replace(dictio_MasVnrType)

In [46]:
df["v_GarageType"].value_counts(1)

attchd     0.603279
detchd     0.328962
builtin    0.044809
otro       0.022951
Name: v_GarageType, dtype: float64

In [47]:
df["v_Electrical"].value_counts(1)

sbrkr    0.899476
fusea    0.074346
otro     0.026178
Name: v_Electrical, dtype: float64

In [48]:
## "v_GarageQual","v_GarageCond"

In [49]:
df["v_GarageQual"].value_counts(1)

ta    0.937705
fa    0.045902
gd    0.010929
po    0.003279
ex    0.002186
Name: v_GarageQual, dtype: float64

In [50]:
df["v_GarageCond"].value_counts(1)

ta    0.951913
fa    0.034973
po    0.006557
gd    0.004372
ex    0.002186
Name: v_GarageCond, dtype: float64

In [51]:
dictio_qual_cond=dict(zip(["ex","gd","ta","fa","po"],[3,3,2,1,1]))

In [52]:
feats_replace_cond_qual=["v_GarageQual","v_GarageCond","v_BsmtCond","v_BsmtQual","v_ExterCond","v_ExterQual","v_HeatingQC","v_KitchenQual"]

In [53]:
for feat in feats_replace_cond_qual:
    print(feat)
    df[feat]=df[feat].replace(dictio_qual_cond)
    print(df[feat].value_counts(1))
    print("\n")

v_GarageQual
2.0    0.937705
1.0    0.049180
3.0    0.013115
Name: v_GarageQual, dtype: float64


v_GarageCond
2.0    0.951913
1.0    0.041530
3.0    0.006557
Name: v_GarageCond, dtype: float64


v_BsmtCond
2.0    0.919700
3.0    0.044968
1.0    0.035332
Name: v_BsmtCond, dtype: float64


v_BsmtQual
3.0    0.493576
2.0    0.471092
1.0    0.035332
Name: v_BsmtQual, dtype: float64


v_ExterCond
2    0.885983
3    0.093096
1    0.020921
Name: v_ExterCond, dtype: float64


v_ExterQual
2    0.648536
3    0.343096
1    0.008368
Name: v_ExterQual, dtype: float64


v_HeatingQC
3    0.676778
2    0.288703
1    0.034519
Name: v_HeatingQC, dtype: float64


v_KitchenQual
2    0.529289
3    0.443515
1    0.027197
Name: v_KitchenQual, dtype: float64




In [54]:
miss=completitud(df)
miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,41,95.711297
1,v_GarageType,41,95.711297
2,v_GarageYrBlt,41,95.711297
3,v_GarageFinish,41,95.711297
4,v_GarageCond,41,95.711297
5,v_BsmtExposure,23,97.594142
6,v_BsmtFinType2,22,97.698745
7,v_BsmtQual,22,97.698745
8,v_BsmtFinType1,22,97.698745
9,v_BsmtCond,22,97.698745


In [55]:
#Remplazamos los valores de la fecha de construccion del garage por la fecha de construcción de la propiedad
df['v_GarageYrBlt']=df['v_GarageYrBlt'].fillna(df['v_YearBuilt']).astype(float)

In [56]:
miss=completitud(df)

miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,41,95.711297
1,v_GarageType,41,95.711297
2,v_GarageFinish,41,95.711297
3,v_GarageCond,41,95.711297
4,v_BsmtExposure,23,97.594142
5,v_BsmtQual,22,97.698745
6,v_BsmtFinType2,22,97.698745
7,v_BsmtCond,22,97.698745
8,v_BsmtFinType1,22,97.698745
9,v_Electrical,1,99.895397


### CATEGORICAS

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
X_train,X_test=train_test_split(df,test_size=.2,random_state=413)

In [59]:
miss=completitud(df)

miss[miss["completitud"]<100]

,columna,total,completitud
0,v_GarageQual,41,95.711297
1,v_GarageType,41,95.711297
2,v_GarageFinish,41,95.711297
3,v_GarageCond,41,95.711297
4,v_BsmtExposure,23,97.594142
5,v_BsmtQual,22,97.698745
6,v_BsmtFinType2,22,97.698745
7,v_BsmtCond,22,97.698745
8,v_BsmtFinType1,22,97.698745
9,v_Electrical,1,99.895397


In [60]:
v_feats=list(filter(lambda x:"v_" in x,list(miss[miss["completitud"]<100]["columna"].values)))

In [61]:
v_feats

['v_GarageQual',
 'v_GarageType',
 'v_GarageFinish',
 'v_GarageCond',
 'v_BsmtExposure',
 'v_BsmtQual',
 'v_BsmtFinType2',
 'v_BsmtCond',
 'v_BsmtFinType1',
 'v_Electrical']

In [62]:
for column in v_feats:
    print(column)
    value=X_train[column].value_counts().index[0]
    print(value)
    print(chi_square(df,column,value))
    print("\n")

v_GarageQual
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageType
attchd
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageFinish
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_GarageCond
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtExposure
no
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtQual
3.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtFinType2
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtCond
2.0
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_BsmtFinType1
unf
Aceptamos HO(La porporción de categorias es la misma que la general)
None


v_Electrical
sbrkr
Aceptamos HO(La porporción de categorias es la misma que la general)
None




In [63]:
from sklearn.impute import SimpleImputer

In [64]:
imp=SimpleImputer(missing_values=np.nan,strategy="most_frequent")

In [65]:
imp.fit(X_train[v_feats])

SimpleImputer(strategy='most_frequent')

In [66]:
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

In [67]:
X_train[v_feats]=pd.DataFrame(imp.transform(X_train[v_feats]),columns=v_feats)

In [68]:
X_test[v_feats]=pd.DataFrame(imp.transform(X_test[v_feats]),columns=v_feats)

In [69]:
X_train["v_GarageYrBlt"]=X_train["v_GarageYrBlt"].astype(int)
X_test["v_GarageYrBlt"]=X_test["v_GarageYrBlt"].astype(int)
X_train["v_YearRemodAdd"]=X_train["v_YearRemodAdd"].astype(int)
X_test["v_YearRemodAdd"]=X_test["v_YearRemodAdd"].astype(int)

## Reduccion de dimensiones

In [70]:
miss=completitud(X_train)

miss[miss["completitud"]<100]

,columna,total,completitud


In [71]:
miss=completitud(X_test)
miss[miss["completitud"]<100]

,columna,total,completitud


In [72]:
X_train["v_MSZoning"].value_counts()

rl      572
rm      137
otro     55
Name: v_MSZoning, dtype: int64

### Relación de valor perdido

In [73]:
X_test["v_MSZoning"].value_counts()

rl      157
rm       24
otro     11
Name: v_MSZoning, dtype: int64

In [74]:
completitud(X_train)

,columna,total,completitud
0,v_MSSubClass,0,100.0
1,v_CentralAir,0,100.0
2,v_Electrical,0,100.0
3,c_1stFlrSF,0,100.0
4,c_2ndFlrSF,0,100.0
...,...,...,...
62,c_BsmtFinSF1,0,100.0
63,v_BsmtFinType2,0,100.0
64,c_BsmtFinSF2,0,100.0
65,v_YearRemodAdd,0,100.0


In [75]:
completitud(X_test)

,columna,total,completitud
0,v_MSSubClass,0,100.0
1,v_CentralAir,0,100.0
2,v_Electrical,0,100.0
3,c_1stFlrSF,0,100.0
4,c_2ndFlrSF,0,100.0
...,...,...,...
62,c_BsmtFinSF1,0,100.0
63,v_BsmtFinType2,0,100.0
64,c_BsmtFinSF2,0,100.0
65,v_YearRemodAdd,0,100.0


### Filtro de baja varianza

In [76]:
X_train.filter(like="c_").describe(percentiles=np.arange(0.1,1.1,.1))

,c_LotFrontage,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,c_1stFlrSF,c_2ndFlrSF,c_GrLivArea,c_BsmtFullBath,c_HalfBath,c_BedroomAbvGr,c_TotRmsAbvGrd,c_GarageCars,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal
count,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.000000,764.0,764.000000
mean,67.167539,9054.078534,77.568063,370.883508,28.454188,582.620419,981.958115,1085.074607,324.500000,1414.069372,0.352094,0.331152,2.820681,6.319372,1.712042,452.715969,77.628272,36.753927,22.515707,2.035340,10.409686,0.0,5.764398
std,18.902681,3968.619060,137.872163,376.470803,111.104637,408.534227,346.601464,293.995241,390.947988,389.155956,0.480670,0.479212,0.660070,1.340993,0.683689,189.226138,103.565179,51.562782,57.891624,18.711931,45.565829,0.0,52.836075
min,21.000000,1300.000000,0.000000,0.000000,0.000000,0.000000,0.000000,480.000000,0.000000,480.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
10%,44.000000,5000.000000,0.000000,0.000000,0.000000,92.000000,622.600000,755.300000,0.000000,912.000000,0.000000,0.000000,2.000000,5.000000,1.000000,240.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
20%,52.000000,6787.200000,0.000000,0.000000,0.000000,209.200000,735.000000,834.200000,0.000000,1056.600000,0.000000,0.000000,2.000000,5.000000,1.000000,288.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
30%,60.000000,7799.500000,0.000000,0.000000,0.000000,315.800000,806.900000,894.000000,0.000000,1175.700000,0.000000,0.000000,3.000000,6.000000,1.000000,352.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
40%,61.000000,8446.000000,0.000000,96.000000,0.000000,414.000000,873.200000,961.000000,0.000000,1276.200000,0.000000,0.000000,3.000000,6.000000,2.000000,429.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,66.000000,9000.000000,0.000000,340.500000,0.000000,523.000000,936.000000,1048.000000,0.000000,1386.000000,0.000000,0.000000,3.000000,6.000000,2.000000,462.000000,0.000000,14.000000,0.000000,0.000000,0.000000,0.0,0.000000
60%,70.000000,9600.000000,0.000000,449.000000,0.000000,646.600000,1029.000000,1115.600000,454.800000,1504.000000,0.000000,0.000000,3.000000,7.000000,2.000000,492.000000,77.200000,35.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [77]:
X_train.drop(columns=["c_PoolArea"],inplace=True)
X_test.drop(columns=["c_PoolArea"],inplace=True)

### Filtro de alta correlación

- La Correlación se utiliza para probar las relaciones entre variables cuantitativas o variables categóricas. En otras palabras, es una medida de cómo se relacionan las cosas. El estudio de cómo se correlacionan las variables se denomina análisis de correlación.
- Un coeficiente de correlación proporciona un resumen numérico del grado de asociación entre dos variables.
- Tipos:
- **Correlación r de Pearson: la correlación r de Pearson es el estadístico de correlación más utilizado para medir el grado de relación entre variables relacionadas linealmente. Por ejemplo, en el mercado de valores, si queremos medir cómo se relacionan dos acciones entre sí, se utiliza la correlación r de Pearson para medir el grado de relación entre las dos. La correlación punto-biserial se realiza con la fórmula de correlación de Pearson excepto que una de las variables es dicotómica.**
- **Correlación de rango de Kendall: la correlación de rango de Kendall es una prueba no paramétrica que mide la fuerza de la dependencia entre dos variables. Si consideramos dos muestras, ayb, donde cada tamaño de muestra es n, sabemos que el número total de emparejamientos con a b es n (n-1) / 2.**
- **Correlación de rango de Spearman: la correlación de rango de Spearman es una prueba no paramétrica que se utiliza para medir el grado de asociación entre dos variables. La prueba de correlación de rango de Spearman no conlleva ninguna suposición sobre la distribución de los datos y es el análisis de correlación apropiado cuando las variables se miden en una escala que es al menos ordinal.**

https://www.google.com/search?q=correlation+between+categorical+and+numerical+variables&rlz=1C1SQJL_esMX974MX980&sxsrf=AOaemvKEG-VYi6gPeh-rCa-8uohkKvAAGw:1642729480236&source=lnms&tbm=isch&sa=X&ved=2ahUKEwj0goaZ3MH1AhUoKEQIHULoBYoQ_AUoAXoECAEQAw&biw=1745&bih=881&dpr=1.1#imgrc=r8MNSNFhNHJGmM

In [78]:
X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

- GrLivArea: superficie habitable por encima del nivel del suelo en pies cuadrados
- TotRmsAbvGrd: total de habitaciones por encima del nivel del suelo (no incluye baños)
- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados

In [79]:
corr=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman")

In [80]:
X_train.filter(like="v_").columns

Index(['v_MSSubClass', 'v_MSZoning', 'v_Street', 'v_LotShape', 'v_LandContour',
       'v_LotConfig', 'v_LandSlope', 'v_Condition1', 'v_Condition2',
       'v_BldgType', 'v_HouseStyle', 'v_OverallQual', 'v_OverallCond',
       'v_YearBuilt', 'v_YearRemodAdd', 'v_RoofStyle', 'v_RoofMatl',
       'v_MasVnrType', 'v_ExterQual', 'v_ExterCond', 'v_Foundation',
       'v_BsmtQual', 'v_BsmtCond', 'v_BsmtExposure', 'v_BsmtFinType1',
       'v_BsmtFinType2', 'v_Heating', 'v_HeatingQC', 'v_CentralAir',
       'v_Electrical', 'v_LowQualFinSF', 'v_BsmtHalfBath', 'v_FullBath',
       'v_KitchenAbvGr', 'v_KitchenQual', 'v_Functional', 'v_Fireplaces',
       'v_GarageType', 'v_GarageYrBlt', 'v_GarageFinish', 'v_GarageQual',
       'v_GarageCond', 'v_PavedDrive'],
      dtype='object')

In [81]:
corr=abs(corr)

In [82]:
for col in corr.columns:
    display(corr[[col]][corr[[col]]>.8].dropna())

,c_LotFrontage
c_LotFrontage,1.0


,c_LotArea
c_LotArea,1.0


,c_MasVnrArea
c_MasVnrArea,1.0


,c_BsmtFinSF1
c_BsmtFinSF1,1.0


,c_BsmtFinSF2
c_BsmtFinSF2,1.0


...

,c_OpenPorchSF
c_OpenPorchSF,1.0


,c_EnclosedPorch
c_EnclosedPorch,1.0


,c_3SsnPorch
c_3SsnPorch,1.0


,c_ScreenPorch
c_ScreenPorch,1.0


,c_MiscVal
c_MiscVal,1.0


In [83]:
X_train.drop(columns=["c_1stFlrSF","c_GarageCars"],inplace=True)
X_test.drop(columns=["c_1stFlrSF","c_GarageCars"],inplace=True)

In [84]:
corr=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman")
corr=abs(corr)

In [85]:
for col in corr.columns:
    display(corr[[col]][corr[[col]]>.8].dropna())

,c_LotFrontage
c_LotFrontage,1.0


,c_LotArea
c_LotArea,1.0


,c_MasVnrArea
c_MasVnrArea,1.0


,c_BsmtFinSF1
c_BsmtFinSF1,1.0


,c_BsmtFinSF2
c_BsmtFinSF2,1.0


...

,c_OpenPorchSF
c_OpenPorchSF,1.0


,c_EnclosedPorch
c_EnclosedPorch,1.0


,c_3SsnPorch
c_3SsnPorch,1.0


,c_ScreenPorch
c_ScreenPorch,1.0


,c_MiscVal
c_MiscVal,1.0


In [86]:
X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_").corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

### Correlación (con el objetivo)

In [87]:
feats=list(X_train.filter(like="c_").columns)+["tgt_SalePrice"]

In [88]:
X_train[feats].corr(method="spearman").iplot(kind="heatmap",colorscale="orrd",title="Matriz de Correlación")

In [89]:
corr=X_train[feats].corr(method="spearman")

In [90]:
corr=abs(corr)

In [91]:
corr[["tgt_SalePrice"]].sort_values(by = 'tgt_SalePrice',ascending = False).style.background_gradient()

,tgt_SalePrice
tgt_SalePrice,1.000000
c_GrLivArea,0.668101
c_GarageArea,0.620256
c_TotalBsmtSF,0.542675
c_OpenPorchSF,0.519157
c_TotRmsAbvGrd,0.477351
c_MasVnrArea,0.441387
c_LotArea,0.362522
c_WoodDeckSF,0.360948
c_HalfBath,0.358473


In [92]:
corr=corr[["tgt_SalePrice"]].sort_values(by = 'tgt_SalePrice',ascending = False)

In [93]:
cols_drop_low_corr=list(corr[corr["tgt_SalePrice"]<.1].index)

In [94]:
cols_drop_low_corr

['c_BsmtFinSF2', 'c_3SsnPorch', 'c_ScreenPorch']

- 3SsnPorch: área de porche de tres estaciones en pies cuadrados
- ScreenPorch: superficie acristalada del porche en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados

In [95]:
X_train=X_train.drop(columns=cols_drop_low_corr)
X_test=X_test.drop(columns=cols_drop_low_corr)

### MULTICOLINEALIDAD

#### MULTICOLINEALIDAD CONTINUAS

In [96]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [97]:
#Función para multicolineaidad con continuas
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [98]:
X_train.drop(columns=['v_YearBuilt','v_YearRemodAdd','v_GarageYrBlt'],inplace=True)
X_test.drop(columns=['v_YearBuilt','v_YearRemodAdd','v_GarageYrBlt'],inplace=True)

In [99]:
for col in X_train.filter(like="c_").columns:
    X_train[col]=X_train[col].astype(float)

In [100]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

In [101]:
vif

,variables,VIF
0,c_LotFrontage,19.683616
1,c_LotArea,8.928313
2,c_MasVnrArea,1.624055
3,c_BsmtFinSF1,25.485288
4,c_BsmtUnfSF,44.434870
5,c_TotalBsmtSF,116.439043
6,c_2ndFlrSF,9.166411
7,c_GrLivArea,97.068657
8,c_BsmtFullBath,2.762614
9,c_HalfBath,2.822937


In [102]:
vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False).head(50)

,variables,VIF
5,c_TotalBsmtSF,116.439043
11,c_TotRmsAbvGrd,72.353109
0,c_LotFrontage,19.683616
7,c_GrLivArea,97.068657
12,c_GarageArea,10.566513
4,c_BsmtUnfSF,44.434870
3,c_BsmtFinSF1,25.485288
10,c_BedroomAbvGr,32.030146


In [103]:
vif_feats=list(vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)["variables"].values)

In [104]:
vif_feats

['c_TotalBsmtSF',
 'c_TotRmsAbvGrd',
 'c_LotFrontage',
 'c_GrLivArea',
 'c_GarageArea',
 'c_BsmtUnfSF',
 'c_BsmtFinSF1',
 'c_BedroomAbvGr']

In [105]:
X_train.drop(columns=["c_TotalBsmtSF"],inplace=True)
X_test.drop(columns=["c_TotalBsmtSF"],inplace=True)

In [106]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in ["tgt_SalePrice"]]])

In [107]:
vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
10,c_TotRmsAbvGrd,71.892122
0,c_LotFrontage,19.647479
6,c_GrLivArea,91.503154
11,c_GarageArea,10.517192
9,c_BedroomAbvGr,31.495250


In [108]:
vif_feats=["c_GrLivArea"]

In [109]:
X_train.drop(columns=vif_feats,inplace=True)
X_test.drop(columns=vif_feats,inplace=True)

In [110]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

vif[vif["VIF"]>7].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
9,c_TotRmsAbvGrd,48.218034
0,c_LotFrontage,19.550756
1,c_LotArea,8.885816
10,c_GarageArea,10.015478
4,c_BsmtUnfSF,7.006906
8,c_BedroomAbvGr,31.488231


In [111]:
vif_feats=list(vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)["variables"].values)

In [112]:
vif_feats

['c_TotRmsAbvGrd', 'c_LotFrontage', 'c_GarageArea', 'c_BedroomAbvGr']

In [113]:
X_train.drop(columns=["c_TotRmsAbvGrd"],inplace=True)
X_test.drop(columns=["c_TotRmsAbvGrd"],inplace=True)

In [114]:
X_train.columns

Index(['v_MSSubClass', 'v_MSZoning', 'c_LotFrontage', 'c_LotArea', 'v_Street',
       'v_LotShape', 'v_LandContour', 'v_LotConfig', 'v_LandSlope',
       'v_Condition1', 'v_Condition2', 'v_BldgType', 'v_HouseStyle',
       'v_OverallQual', 'v_OverallCond', 'v_RoofStyle', 'v_RoofMatl',
       'v_MasVnrType', 'c_MasVnrArea', 'v_ExterQual', 'v_ExterCond',
       'v_Foundation', 'v_BsmtQual', 'v_BsmtCond', 'v_BsmtExposure',
       'v_BsmtFinType1', 'c_BsmtFinSF1', 'v_BsmtFinType2', 'c_BsmtUnfSF',
       'v_Heating', 'v_HeatingQC', 'v_CentralAir', 'v_Electrical',
       'c_2ndFlrSF', 'v_LowQualFinSF', 'c_BsmtFullBath', 'v_BsmtHalfBath',
       'v_FullBath', 'c_HalfBath', 'c_BedroomAbvGr', 'v_KitchenAbvGr',
       'v_KitchenQual', 'v_Functional', 'v_Fireplaces', 'v_GarageType',
       'v_GarageFinish', 'c_GarageArea', 'v_GarageQual', 'v_GarageCond',
       'v_PavedDrive', 'c_WoodDeckSF', 'c_OpenPorchSF', 'c_EnclosedPorch',
       'c_MiscVal', 'tgt_SalePrice'],
      dtype='object')

In [115]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])

vif_feats=list(vif[vif["VIF"]>7].sort_values(by=["VIF"],ascending=False)["variables"][:10].values)

In [116]:


vif[vif["VIF"]>10].sort_values(by=["variables","VIF"],ascending=False)

,variables,VIF
0,c_LotFrontage,18.642185
8,c_BedroomAbvGr,18.346209


In [117]:
X_train.drop(columns=["c_LotFrontage"],inplace=True)
X_test.drop(columns=["c_LotFrontage"],inplace=True)

In [118]:
#BsmtCond: Estado general del sótano
#ExterCond: Estado actual del material en el exterior

In [119]:
vif=calc_vif(X_train[[x for x in X_train.filter(like="c_") if x not in [ "tgt_SalePrice"]]])
vif[vif["VIF"]>7].sort_values(by=["VIF"],ascending=False)

,variables,VIF
7,c_BedroomAbvGr,13.880824
8,c_GarageArea,9.217258
0,c_LotArea,7.116685


In [120]:
X_train

,v_MSSubClass,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal,tgt_SalePrice
0,80,rl,7134.0,pave,reg,otro,inside,otro,norm,norm,1fam,otro,5,5,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,no,alq,384.0,unf,0.0,gasa,2,y,sbrkr,0.0,0,0.0,0,1,0.0,3.0,1,2,otro,1,detchd,unf,572.0,2.0,2.0,y,0.0,50.0,0.0,0.0,130000
1,50,rm,6240.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1 5fin,6,6,gable,compshg,none,0.0,2,2,pconc,2.0,2.0,no,unf,0.0,unf,816.0,gasa,2,y,sbrkr,0.0,360,0.0,0,1,0.0,3.0,1,2,typ,1,detchd,unf,528.0,2.0,2.0,y,112.0,0.0,0.0,400.0,114500
2,20,rm,6000.0,pave,reg,otro,inside,otro,norm,norm,otro,1story,4,4,gable,compshg,none,0.0,1,2,cblock,1.0,2.0,no,unf,0.0,unf,936.0,gasa,2,n,sbrkr,0.0,0,0.0,0,1,0.0,2.0,1,2,otro,0,detchd,unf,576.0,2.0,2.0,y,0.0,32.0,112.0,0.0,93000
3,20,rl,9937.0,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,5,7,hip,compshg,none,0.0,2,3,pconc,2.0,2.0,no,otro,637.0,unf,849.0,gasa,3,y,sbrkr,0.0,0,1.0,0,1,0.0,3.0,1,2,typ,0,detchd,fin,480.0,2.0,2.0,y,0.0,0.0,0.0,0.0,167000
4,70,rm,17671.0,pave,reg,lvl,corner,gtl,otro,norm,1fam,2story,8,9,gable,compshg,none,0.0,3,3,otro,2.0,2.0,no,otro,216.0,unf,700.0,gasa,3,y,sbrkr,826.0,0,0.0,0,1,1.0,4.0,1,3,typ,1,attchd,unf,424.0,2.0,2.0,p,0.0,169.0,0.0,0.0,168000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,30,rl,5400.0,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,5,6,gable,compshg,none,0.0,2,2,otro,1.0,2.0,no,unf,0.0,unf,691.0,gasa,3,y,fusea,0.0,0,0.0,0,1,0.0,2.0,1,3,typ,0,detchd,unf,216.0,1.0,2.0,n,0.0,20.0,94.0,0.0,86000
760,20,otro,8640.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,7,5,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,no,unf,0.0,unf,1372.0,gasa,3,y,sbrkr,0.0,0,0.0,0,2,0.0,3.0,1,3,typ,0,attchd,fin,529.0,2.0,2.0,y,0.0,140.0,0.0,0.0,250580
761,20,rl,7763.0,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,7,gable,compshg,none,0.0,2,3,cblock,2.0,2.0,no,otro,504.0,otro,319.0,gasa,2,y,sbrkr,0.0,0,1.0,0,1,0.0,3.0,1,2,typ,0,detchd,unf,506.0,2.0,2.0,y,0.0,0.0,0.0,0.0,140000
762,120,rm,5587.0,pave,otro,otro,inside,otro,norm,norm,otro,1story,8,5,hip,compshg,otro,186.0,3,2,pconc,3.0,2.0,otro,glq,1480.0,unf,120.0,gasa,3,y,sbrkr,0.0,0,1.0,1,2,0.0,2.0,1,3,typ,1,attchd,fin,482.0,2.0,2.0,y,162.0,53.0,0.0,0.0,392500


#### MULTICOLINEALIDAD CATEGORICAS

In [122]:
## Multicolinealidad Categoricas##
def multicolinealidad_cate(df,y,threshold=5,only_final_vif=False):
    '''
    Función para calcular el VIF Genralizado, (GVIF, Fox and Monette 1992)
    Las variables no deben de deben de estar transformadas con OneHotEncoder ya que la función lo realiza internamnete.
    Args:
        df
        y
        threshold
        only_final_vif
    
    Returns:
        pandas data frame: a data frame, indexed by factor of the GVIF, GVIF^(1/2Df), VIF^(1/2Df)^2 
    '''

    df_x = df.drop(y,axis=1)
    #Guardamos los nombres de las columnas añadiendo el prefijo

    onehot_list = list(df_x.select_dtypes(include=['category', 'object', 'string']).columns)
    #Generamos las variables dummy 
    
    df_1hot = pd.get_dummies(df_x, drop_first=True, dummy_na=False, prefix_sep='_')


    #Dataframe vacio para guardar los resultados
    gvif_df = pd.DataFrame(columns = ['factor','GVIF','Df','GVIF^(1/2Df)', 'VIF'])

    # Iteramos sobre las columas
    for (columnName, columnData) in df_x.iteritems():
        #Si se generaron las de una columna, es decir la variable tiene más de dos posibilidades utilizamos todas las variables creadas
        
        if columnName in onehot_list:
            X1 = df_1hot.loc[:, df_1hot.columns.str.startswith(columnName)]
            X2 = df_1hot.loc[:, ~df_1hot.columns.str.startswith(columnName)]
        else:
            X1 = df_1hot[[columnName]].values
            X2 = df_1hot.loc[:, df_1hot.columns != columnName].values
        #display(X1)
        # Calculamose gvif
        #A la matriz de correlación de las variables codificadas en caliente del atributo en consideración.
        #B la matriz de correlación de todos los demás atributos del conjunto de datos (uno codificado en dummy y numérico) excluyendo los de A.
        #C la matriz de correlación de las variables consideradas tanto en A como en B.
        #GVIF= (det(A)*det(B))/det(C)
        #display(np.corrcoef(X1, rowvar=False))
        gvif = np.linalg.det(np.array(np.corrcoef(X1, rowvar=False), ndmin=2)) * np.linalg.det(np.corrcoef(X2, rowvar=False)) / np.linalg.det(np.corrcoef(np.append(X1, X2, axis=1), rowvar=False))
        #(GVIF) elevado a (1 / (2 * grados de libertad))
        #Los grados de libertad es 1-n (n=Número de opciones que tiene la variable categorica)
        gvif_12df = np.power(gvif, 1 / (2 * X1.shape[1]))
        gvif_12df_sq = gvif_12df ** 2
        DF =  X1.shape[1]
    

        # 
        new_row = {'factor': columnName, 'GVIF': gvif,'Df':DF, 'GVIF^(1/2Df)': gvif_12df, 'VIF': gvif_12df_sq}
        gvif_df = gvif_df.append(new_row, ignore_index=True)

    gvif_df = gvif_df.set_index('factor')
    if only_final_vif:
        gvif_df_final = gvif_df.drop(['GVIF','Df','GVIF^(1/2Df)'],axis=1)
    else:
        gvif_df_final = gvif_df 
    gvif_filter = gvif_df.loc[gvif_df['VIF'] >= threshold]['VIF'].to_dict()
    if gvif_filter:
        for i in gvif_filter.keys():
            df_x_m = df_x.drop([i],axis=1)
    else:
        df_x_m = df_x
    df_m=pd.concat([df_x_m,df[y]],axis=1)
    return gvif_df_final

In [123]:
v_feats=list(X_train.filter(like="v_").columns)

In [124]:
results=multicolinealidad_cate(df=X_train[v_feats+["tgt_SalePrice"]],y="tgt_SalePrice")

In [127]:
X_train["v_BldgType"].value_counts()

1fam    631
otro    133
Name: v_BldgType, dtype: int64

In [125]:
X_train["v_OverallQual"].value_counts()

5     232
6     190
7     186
8      72
4      57
3      12
9      11
10      2
2       2
Name: v_OverallQual, dtype: int64

In [126]:
X_train["v_OverallCond"].value_counts()

5    434
6    136
7    108
8     39
4     27
9      9
3      8
2      3
Name: v_OverallCond, dtype: int64

In [127]:
X_train[["v_MSSubClass","v_BldgType"]].head(10)

,v_MSSubClass,v_BldgType
0,80,1fam
1,50,1fam
2,20,otro
3,20,1fam
4,70,1fam
5,180,otro
6,80,1fam
7,20,1fam
8,50,1fam
9,50,1fam


In [128]:
results

,GVIF,Df,GVIF^(1/2Df),VIF
factor,,,,
v_MSSubClass,17.302375,1,4.159612,17.302375
v_MSZoning,2.443140,2,1.250222,1.563055
v_Street,1.252270,1,1.119049,1.252270
v_LotShape,1.434892,1,1.197870,1.434892
v_LandContour,1.641006,1,1.281018,1.641006
v_LotConfig,1.344334,2,1.076779,1.159454
v_LandSlope,1.614267,1,1.270538,1.614267
v_Condition1,1.286659,1,1.134310,1.286659
v_Condition2,1.783721,1,1.335560,1.783721


In [129]:
X_train=X_train.drop(columns=["v_MSSubClass","v_OverallQual"])
X_test=X_test.drop(columns=["v_MSSubClass","v_OverallQual"])

In [130]:
v_feats=list(X_train.filter(like="v_").columns)

In [131]:
results=multicolinealidad_cate(df=X_train[v_feats+["tgt_SalePrice"]],y="tgt_SalePrice")

In [132]:
results

,GVIF,Df,GVIF^(1/2Df),VIF
factor,,,,
v_MSZoning,2.344668,2,1.237429,1.531231
v_Street,1.234622,1,1.111136,1.234622
v_LotShape,1.401719,1,1.183942,1.401719
v_LandContour,1.571198,1,1.253474,1.571198
v_LotConfig,1.302931,2,1.068391,1.141460
v_LandSlope,1.574798,1,1.254910,1.574798
v_Condition1,1.260128,1,1.122554,1.260128
v_Condition2,1.731788,1,1.315974,1.731788
v_BldgType,1.740818,1,1.319401,1.740818


In [133]:
X_train=X_train.drop(columns=["v_BsmtHalfBath"])
X_test=X_test.drop(columns=["v_BsmtHalfBath"])

In [134]:
##CREAR LAS VARIABLES DUMMY

In [135]:
from sklearn.preprocessing import OneHotEncoder

In [136]:
dummy_vars=["v_MSZoning","v_Street","v_LotShape","v_LandContour","v_LotConfig","v_LandSlope","v_Condition1","v_Condition2","v_BldgType","v_HouseStyle",
"v_RoofStyle","v_MasVnrType","v_Foundation","v_BsmtExposure","v_BsmtFinType1","v_BsmtFinType2","v_Heating","v_CentralAir",
"v_Electrical","v_Functional","v_GarageType","v_GarageFinish","v_PavedDrive","v_RoofMatl","v_OverallCond"]

In [137]:
X_train.shape

(764, 51)

In [138]:
X_test.shape

(192, 51)

In [139]:
X_test

,v_MSZoning,c_LotArea,v_Street,v_LotShape,v_LandContour,v_LotConfig,v_LandSlope,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallCond,v_RoofStyle,v_RoofMatl,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtUnfSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_2ndFlrSF,v_LowQualFinSF,c_BsmtFullBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,v_Functional,v_Fireplaces,v_GarageType,v_GarageFinish,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal,tgt_SalePrice
0,rl,7677,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,no,otro,570,unf,203,gasa,3,y,sbrkr,0,0,0,1,0,2,1,2,typ,0,attchd,unf,240,2.0,2.0,y,0,0,0,0,110000
1,rl,8899,pave,otro,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,none,0.0,3,2,pconc,3.0,2.0,otro,glq,24,unf,1316,gasa,3,y,sbrkr,0,0,0,2,0,3,1,3,typ,0,attchd,fin,396,2.0,2.0,y,100,30,0,0,181134
2,rl,12640,pave,otro,lvl,inside,gtl,norm,norm,otro,1story,5,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,otro,otro,936,otro,396,gasa,2,y,sbrkr,0,0,0,2,0,4,2,2,typ,0,attchd,unf,574,2.0,2.0,y,40,0,0,0,150900
3,otro,9000,pave,reg,lvl,corner,gtl,norm,norm,1fam,otro,3,gable,compshg,none,0.0,2,2,otro,1.0,1.0,no,unf,0,unf,784,gasa,2,n,fusea,0,0,0,1,0,2,1,2,typ,0,detchd,unf,360,1.0,1.0,n,0,0,91,0,76000
4,rl,12090,pave,reg,lvl,inside,gtl,norm,norm,1fam,otro,7,hip,compshg,brkface,74.0,2,2,cblock,3.0,2.0,no,unf,0,unf,585,gasa,3,y,sbrkr,728,0,0,3,1,3,1,2,typ,1,builtin,fin,477,2.0,2.0,y,268,112,0,0,178000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,rl,7094,pave,otro,lvl,inside,gtl,norm,norm,1fam,1story,5,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,no,otro,180,otro,340,gasa,2,y,sbrkr,0,0,0,1,0,3,1,2,typ,0,detchd,rfn,384,2.0,2.0,y,0,0,0,0,125000
188,rl,6420,pave,otro,lvl,inside,gtl,norm,norm,1fam,1story,7,gable,compshg,none,0.0,2,2,pconc,3.0,3.0,otro,otro,210,otro,219,gasa,1,y,fusea,0,0,0,1,0,2,1,2,typ,0,detchd,unf,308,2.0,2.0,y,0,0,0,0,123500
189,rl,7388,pave,reg,lvl,corner,gtl,norm,norm,1fam,1story,6,gable,compshg,none,0.0,2,2,cblock,2.0,2.0,no,otro,405,unf,658,gasa,3,y,sbrkr,0,0,1,1,0,3,1,3,typ,0,detchd,unf,624,2.0,2.0,y,0,0,0,0,150750
190,rl,8700,pave,reg,lvl,inside,gtl,norm,norm,1fam,1story,6,hip,compshg,brkface,148.0,2,3,cblock,2.0,2.0,otro,alq,776,unf,344,gasa,3,y,sbrkr,0,0,1,2,0,3,1,2,typ,0,attchd,rfn,525,2.0,2.0,y,192,20,123,0,155000


In [140]:
one=OneHotEncoder(drop="first",handle_unknown ="error")
#drop="first"

one.fit(X_train[dummy_vars])

replace_name=dict(zip(list(map(lambda x:f"x{str(x)}_",range(len(dummy_vars)))),dummy_vars))

features=one.get_feature_names()

features=list(map(lambda x:multiple_replace(x, replace_name),features))

X_train[features]=pd.DataFrame(one.transform(X_train[dummy_vars]).toarray(),columns=features)
X_test[features]=pd.DataFrame(one.transform(X_test[dummy_vars]).toarray(),columns=features)

X_train.drop(columns=dummy_vars,inplace=True)
X_test.drop(columns=dummy_vars,inplace=True)

In [141]:
X_train

,c_LotArea,c_MasVnrArea,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,c_BsmtFinSF1,c_BsmtUnfSF,v_HeatingQC,c_2ndFlrSF,v_LowQualFinSF,c_BsmtFullBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,c_GarageArea,v_GarageQual,v_GarageCond,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal,tgt_SalePrice,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,7134.0,0.0,2,2,2.0,2.0,384.0,0.0,2,0.0,0,0.0,1,0.0,3.0,1,2,1,572.0,2.0,2.0,0.0,50.0,0.0,0.0,130000,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,6240.0,0.0,2,2,2.0,2.0,0.0,816.0,2,0.0,360,0.0,1,0.0,3.0,1,2,1,528.0,2.0,2.0,112.0,0.0,0.0,400.0,114500,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,6000.0,0.0,1,2,1.0,2.0,0.0,936.0,2,0.0,0,0.0,1,0.0,2.0,1,2,0,576.0,2.0,2.0,0.0,32.0,112.0,0.0,93000,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,9937.0,0.0,2,3,2.0,2.0,637.0,849.0,3,0.0,0,1.0,1,0.0,3.0,1,2,0,480.0,2.0,2.0,0.0,0.0,0.0,0.0,167000,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,17671.0,0.0,3,3,2.0,2.0,216.0,700.0,3,826.0,0,0.0,1,1.0,4.0,1,3,1,424.0,2.0,2.0,0.0,169.0,0.0,0.0,168000,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,5400.0,0.0,2,2,1.0,2.0,0.0,691.0,3,0.0,0,0.0,1,0.0,2.0,1,3,0,216.0,1.0,2.0,0.0,20.0,94.0,0.0,86000,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
760,8640.0,0.0,3,2,3.0,2.0,0.0,1372.0,3,0.0,0,0.0,2,0.0,3.0,1,3,0,529.0,2.0,2.0,0.0,140.0,0.0,0.0,250580,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,7763.0,0.0,2,3,2.0,2.0,504.0,319.0,2,0.0,0,1.0,1,0.0,3.0,1,2,0,506.0,2.0,2.0,0.0,0.0,0.0,0.0,140000,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
762,5587.0,186.0,3,2,3.0,2.0,1480.0,120.0,3,0.0,0,1.0,2,0.0,2.0,1,3,1,482.0,2.0,2.0,162.0,53.0,0.0,0.0,392500,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### PCA

In [142]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

**Necesitamos normalizar los datos antes del análisis de componentes principales (PCA)**
- El PCA calcula una nueva proyección de su conjunto de datos. Y el nuevo eje se basa en la desviación estándar de sus variables. Por tanto, una variable con una desviación estándar alta tendrá un peso mayor para el cálculo del eje que una variable con una desviación estándar baja. Si normaliza sus datos, todas las variables tienen la misma desviación estándar, por lo que todas las variables tienen el mismo peso y su PCA calcula el eje relevante.

In [143]:
X_train.shape

(764, 71)

In [144]:
X_train.filter(like="c_")

,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_BsmtFullBath,c_HalfBath,c_BedroomAbvGr,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal
0,7134.0,0.0,384.0,0.0,0.0,0.0,0.0,3.0,572.0,0.0,50.0,0.0,0.0
1,6240.0,0.0,0.0,816.0,0.0,0.0,0.0,3.0,528.0,112.0,0.0,0.0,400.0
2,6000.0,0.0,0.0,936.0,0.0,0.0,0.0,2.0,576.0,0.0,32.0,112.0,0.0
3,9937.0,0.0,637.0,849.0,0.0,1.0,0.0,3.0,480.0,0.0,0.0,0.0,0.0
4,17671.0,0.0,216.0,700.0,826.0,0.0,1.0,4.0,424.0,0.0,169.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,5400.0,0.0,0.0,691.0,0.0,0.0,0.0,2.0,216.0,0.0,20.0,94.0,0.0
760,8640.0,0.0,0.0,1372.0,0.0,0.0,0.0,3.0,529.0,0.0,140.0,0.0,0.0
761,7763.0,0.0,504.0,319.0,0.0,1.0,0.0,3.0,506.0,0.0,0.0,0.0,0.0
762,5587.0,186.0,1480.0,120.0,0.0,1.0,0.0,2.0,482.0,162.0,53.0,0.0,0.0


In [145]:
scaler = StandardScaler()
scaler.fit(X_train.filter(like="c_"))
df_train_sc=pd.DataFrame(scaler.transform(X_train.filter(like="c_")))
df_test_sc=pd.DataFrame(scaler.transform(X_test.filter(like="c_")))

In [146]:
df_train_sc

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.484132,-0.562977,0.034863,-1.427058,-0.830577,-0.732986,-0.691487,0.271845,0.630791,-0.750051,0.257060,-0.389183,-0.109171
1,-0.709547,-0.562977,-0.985804,0.571635,-0.830577,-0.732986,-0.691487,0.271845,0.398113,0.332102,-0.713266,-0.389183,7.466373
2,-0.770061,-0.562977,-0.985804,0.865560,-0.830577,-0.732986,-0.691487,-1.244138,0.651944,-0.750051,-0.092257,1.546733,-0.109171
3,0.222621,-0.562977,0.707335,0.652465,-0.830577,1.348804,-0.691487,0.271845,0.144282,-0.750051,-0.713266,-0.389183,-0.109171
4,2.172687,-0.562977,-0.411679,0.287507,1.283620,-0.732986,1.396639,1.787829,-0.151854,-0.750051,2.566438,-0.389183,-0.109171
...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,-0.921346,-0.562977,-0.985804,0.265463,-0.830577,-0.732986,-0.691487,-1.244138,-1.251788,-0.750051,-0.325136,1.235604,-0.109171
760,-0.104407,-0.562977,-0.985804,1.933490,-0.830577,-0.732986,-0.691487,0.271845,0.403401,-0.750051,2.003649,-0.389183,-0.109171
761,-0.325535,-0.562977,0.353822,-0.645706,-0.830577,1.348804,-0.691487,0.271845,0.281774,-0.750051,-0.713266,-0.389183,-0.109171
762,-0.874196,0.786982,2.948019,-1.133133,-0.830577,1.348804,-0.691487,-1.244138,0.154858,0.815206,0.315280,-0.389183,-0.109171


In [147]:
df_train_sc.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12
count,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,7.640000e+02,764.000000,7.640000e+02
mean,-1.441546e-16,-2.325074e-17,-9.997820e-17,-5.347671e-17,3.720119e-17,-8.951536e-17,-3.720119e-17,-1.395045e-16,8.602775e-17,4.650149e-18,-8.137760e-17,0.000000,4.650149e-18
std,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655e+00,1.000655,1.000655e+00
min,-1.955128e+00,-5.629771e-01,-9.858041e-01,-1.427058e+00,-8.305774e-01,-7.329864e-01,-6.914866e-01,-2.760122e+00,-2.394027e+00,-7.500506e-01,-7.132665e-01,-0.389183,-1.091711e-01
25%,-4.674909e-01,-5.629771e-01,-9.858041e-01,-7.522543e-01,-8.305774e-01,-7.329864e-01,-6.914866e-01,-1.244138e+00,-7.652789e-01,-7.500506e-01,-7.132665e-01,-0.389183,-1.091711e-01
50%,-1.363546e-02,-5.629771e-01,-8.075902e-02,-1.460330e-01,-8.305774e-01,-7.329864e-01,-6.914866e-01,2.718452e-01,4.909530e-02,-7.500506e-01,-4.415749e-01,-0.389183,-1.091711e-01
75%,3.645774e-01,3.079645e-01,7.192955e-01,5.697980e-01,9.662342e-01,1.348804e+00,1.396639e+00,2.718452e-01,5.898080e-01,6.606131e-01,3.929062e-01,-0.389183,-1.091711e-01
max,1.555890e+01,7.035989e+00,3.192554e+00,3.070001e+00,2.676021e+00,3.430594e+00,3.484764e+00,3.303812e+00,3.026320e+00,3.510927e+00,5.904363e+00,5.314857,1.125415e+01


In [148]:
for n_compo in range(1,11):
    print(f"N componentes : {n_compo}")
    pca = PCA(n_components=n_compo)
    pca.fit(df_train_sc)
    print(sum(pca.explained_variance_ratio_))
    print("\n")

N componentes : 1
0.1828802115960549


N componentes : 2
0.36351003427305545


N componentes : 3
0.47467953418484066


N componentes : 4
0.5639748761529716


N componentes : 5
0.6415618405129468


N componentes : 6
0.7082925942972925


N componentes : 7
0.7742966058115824


N componentes : 8
0.8370893467893612


N componentes : 9
0.8837160387458455


N componentes : 10
0.9277316566749121




In [149]:
pca = PCA(n_components=8)
pca.fit(df_train_sc)

PCA(n_components=8)

In [150]:
X_train_pca=pd.DataFrame(pca.transform(df_train_sc))
X_test_pca=pd.DataFrame(pca.transform(df_test_sc))

In [151]:
#Porcentaje de varianza explicada por cada uno de los componentes seleccionados.
pca.explained_variance_ratio_

array([0.18288021, 0.18062982, 0.1111695 , 0.08929534, 0.07758696,
       0.06673075, 0.06600401, 0.06279274])

In [152]:
sum(pca.explained_variance_ratio_)

0.8370893467893612

In [153]:
X_train_pca

,0,1,2,3,4,5,6,7
0,-0.096760,-0.811561,-0.111806,-0.371113,0.047203,-0.202982,0.677904,0.470138
1,-1.476732,-0.846759,1.183658,2.329822,6.584931,0.901212,-1.489733,1.551939
2,-1.912472,-0.884430,1.064054,-0.691165,-0.402300,1.223932,-0.699799,-0.219011
3,0.372751,-1.174484,0.455268,0.505999,-0.626337,0.023592,0.611548,0.829610
4,-0.019905,3.237667,-0.397981,0.444356,-0.025801,1.144524,2.378782,0.550505
...,...,...,...,...,...,...,...,...
759,-2.346381,-1.531437,-0.030971,-0.731507,-0.200946,0.414375,-0.453024,-0.240067
760,-1.328175,0.753412,2.227293,-0.949184,-0.068534,0.905871,1.087440,0.304919
761,0.532111,-1.484325,-0.261021,0.138629,-0.394182,-0.140288,0.492640,0.751395
762,2.979665,-2.335322,-0.096099,-0.914821,0.061197,-0.019523,-0.529044,-0.234982


In [155]:
X_train.filter(like="v_")

,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,v_HeatingQC,v_LowQualFinSF,v_FullBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,v_GarageQual,v_GarageCond,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,2,2,2.0,2.0,2,0,1,1,2,1,2.0,2.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2,2,2.0,2.0,2,360,1,1,2,1,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,2,1.0,2.0,2,0,1,1,2,0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2,3,2.0,2.0,3,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,3,3,2.0,2.0,3,0,1,1,3,1,2.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,2,2,1.0,2.0,3,0,1,1,3,0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
760,3,2,3.0,2.0,3,0,2,1,3,0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,2,3,2.0,2.0,2,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
762,3,2,3.0,2.0,3,0,2,1,3,1,2.0,2.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [160]:
X_train.reset_index(drop=True).filter(like="v_")

,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,v_HeatingQC,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,v_GarageQual,v_GarageCond,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,2,2,2.0,2.0,3,0,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2,2,2.0,2.0,3,0,0,2,3,2,0,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,2,3.0,2.0,2,0,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,2,3.0,2.0,3,0,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,2,2,1.0,2.0,3,0,0,1,1,3,0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,2,3,2.0,1.0,2,0,0,1,1,3,2,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
760,3,2,3.0,2.0,3,0,0,2,1,3,1,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,3,2,3.0,2.0,3,0,0,2,1,3,1,2.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
762,2,3,2.0,2.0,3,0,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [156]:
pd.concat([X_train_pca,X_train.reset_index(drop=True).filter(like="v_")],axis=1)

,0,1,2,3,4,5,6,7,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,v_HeatingQC,v_LowQualFinSF,v_FullBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,v_GarageQual,v_GarageCond,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,-0.096760,-0.811561,-0.111806,-0.371113,0.047203,-0.202982,0.677904,0.470138,2,2,2.0,2.0,2,0,1,1,2,1,2.0,2.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-1.476732,-0.846759,1.183658,2.329822,6.584931,0.901212,-1.489733,1.551939,2,2,2.0,2.0,2,360,1,1,2,1,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-1.912472,-0.884430,1.064054,-0.691165,-0.402300,1.223932,-0.699799,-0.219011,1,2,1.0,2.0,2,0,1,1,2,0,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.372751,-1.174484,0.455268,0.505999,-0.626337,0.023592,0.611548,0.829610,2,3,2.0,2.0,3,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.019905,3.237667,-0.397981,0.444356,-0.025801,1.144524,2.378782,0.550505,3,3,2.0,2.0,3,0,1,1,3,1,2.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,-2.346381,-1.531437,-0.030971,-0.731507,-0.200946,0.414375,-0.453024,-0.240067,2,2,1.0,2.0,3,0,1,1,3,0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
760,-1.328175,0.753412,2.227293,-0.949184,-0.068534,0.905871,1.087440,0.304919,3,2,3.0,2.0,3,0,2,1,3,0,2.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,0.532111,-1.484325,-0.261021,0.138629,-0.394182,-0.140288,0.492640,0.751395,2,3,2.0,2.0,2,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
762,2.979665,-2.335322,-0.096099,-0.914821,0.061197,-0.019523,-0.529044,-0.234982,3,2,3.0,2.0,3,0,2,1,3,1,2.0,2.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [157]:
pd.concat([X_test_pca,X_test.reset_index(drop=True).filter(like="v_")],axis=1)

,0,1,2,3,4,5,6,7,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,v_HeatingQC,v_LowQualFinSF,v_FullBath,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,v_GarageQual,v_GarageCond,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,-0.745735,-2.036841,-0.472391,-0.417851,0.156645,-0.819586,0.550112,0.215250,2,2,2.0,2.0,3,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,-1.563810,0.061636,1.634353,0.277245,-0.124665,-0.484243,0.277239,-0.345820,3,2,3.0,2.0,3,0,2,1,3,0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.579334,-0.291562,0.046778,1.705541,-0.678611,-0.597759,0.762837,0.786301,2,2,2.0,2.0,2,0,2,2,2,0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-2.094194,-1.241845,0.487897,0.015947,-0.411750,0.492198,-0.265421,-0.075931,2,2,1.0,1.0,2,0,1,1,2,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.369616,2.367432,0.102400,-0.015254,0.761138,0.020345,0.502313,-1.727871,2,2,3.0,2.0,3,0,3,1,2,1,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-1.128394,-0.950614,-0.135360,0.027680,-0.076323,-0.864033,0.369667,0.459710,2,2,2.0,2.0,2,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
188,-1.192705,-1.772963,-0.216829,-0.667600,0.257548,-0.821973,0.373252,0.125842,2,2,3.0,3.0,1,0,1,1,2,0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
189,0.340278,-1.084573,0.483624,0.058467,-0.460590,0.036058,0.311486,0.743192,2,2,2.0,2.0,3,0,1,1,3,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
190,1.418662,-1.331050,-0.209349,0.881927,-0.881547,1.073140,-1.408966,-0.594630,2,3,2.0,2.0,3,0,2,1,2,0,2.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [158]:
#Para visualización creamos dos o tres componentes para tener una idea de como se ve nuestra información continua
#Tomando en cuenta la varianza explicada (algunas veces no es tan representativo)
pca = PCA(n_components=2)
dimention_2=pca.fit_transform(df_train_sc)
total_var = pca.explained_variance_ratio_.sum() * 100
fig = px.scatter(dimention_2, x=0, y=1,title=f'Total de Varianza Explicada: {total_var:.2f}%',
                 labels={'0': 'Componente Principal 1', '1': 'Componente Principal 2'})
fig.show()

In [160]:
pca = PCA(n_components=3)
dimention_3=pca.fit_transform(df_train_sc)

total_var = pca.explained_variance_ratio_.sum() * 100

fig = px.scatter_3d(
    dimention_3, x=0, y=1, z=2, 
    title=f'Total de Varianza Explicada: {total_var:.2f}%',
    labels={'0': 'CP 1', '1': 'CP 2', '2': 'CP 3'}
)
fig.show()

### K- Best

Score function:
- For regression: f_regression, mutual_info_regression
- For classification: chi2, f_classif, mutual_info_classif

**f_regression**
- Pruebas de regresión lineal univariante.

- Modelo lineal para probar el efecto individual de cada uno de los muchos regresores. Esta es una función de puntuación que se utilizará en un procedimiento de selección de características

- Esto se hace en 2 pasos:

- Se calcula la correlación entre cada regresor y el objetivo, es decir, ((X [:, i] - mean (X [:, i])) * (y - mean_y)) / (std (X [:, i] ) * estándar (y)).

- Se convierte en una puntuación F y luego en un valor p.

- mutual_info_classif : para discretas y continuas
- f_regressión : solo continuas

https://towardsdatascience.com/fisher-test-for-regression-analysis-1e1687867259

** Estime la información mutua para una variable objetivo discreta.

** La información mutua (MI) [1] entre dos variables aleatorias es un valor no negativo, que mide la dependencia entre las variables. Es igual a cero si y solo si dos variables aleatorias son independientes, y los valores más altos significan una mayor dependencia.

** La función se basa en métodos no paramétricos basados ​​en la estimación de la entropía a partir de distancias de los vecinos más cercanos k.

In [211]:
X=X_train[[x for x in X_train if x!="tgt_SalePrice"]].filter(like="c_")
y=X_train[[x for x in X_train if x=="tgt_SalePrice"]]

In [219]:
from sklearn.feature_selection import SelectKBest,mutual_info_classif,mutual_info_regression

In [221]:
kb = SelectKBest(mutual_info_regression , k=5)

In [222]:
X

,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_BsmtFullBath,c_HalfBath,c_BedroomAbvGr,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal
0,7134.0,0.0,384.0,0.0,0.0,0.0,0.0,3.0,572.0,0.0,50.0,0.0,0.0
1,6240.0,0.0,0.0,816.0,0.0,0.0,0.0,3.0,528.0,112.0,0.0,0.0,400.0
2,6000.0,0.0,0.0,936.0,0.0,0.0,0.0,2.0,576.0,0.0,32.0,112.0,0.0
3,9937.0,0.0,637.0,849.0,0.0,1.0,0.0,3.0,480.0,0.0,0.0,0.0,0.0
4,17671.0,0.0,216.0,700.0,826.0,0.0,1.0,4.0,424.0,0.0,169.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,5400.0,0.0,0.0,691.0,0.0,0.0,0.0,2.0,216.0,0.0,20.0,94.0,0.0
760,8640.0,0.0,0.0,1372.0,0.0,0.0,0.0,3.0,529.0,0.0,140.0,0.0,0.0
761,7763.0,0.0,504.0,319.0,0.0,1.0,0.0,3.0,506.0,0.0,0.0,0.0,0.0
762,5587.0,186.0,1480.0,120.0,0.0,1.0,0.0,2.0,482.0,162.0,53.0,0.0,0.0


In [223]:
X_new=kb.fit_transform(X, y)

In [224]:
cols=X.columns[kb.get_support()]

In [225]:
pd.DataFrame(X_new,columns=cols)

,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_GarageArea,c_OpenPorchSF
0,384.0,0.0,0.0,572.0,50.0
1,0.0,816.0,0.0,528.0,0.0
2,0.0,936.0,0.0,576.0,32.0
3,637.0,849.0,0.0,480.0,0.0
4,216.0,700.0,826.0,424.0,169.0
...,...,...,...,...,...
759,0.0,691.0,0.0,216.0,20.0
760,0.0,1372.0,0.0,529.0,140.0
761,504.0,319.0,0.0,506.0,0.0
762,1480.0,120.0,0.0,482.0,53.0


In [226]:
scores=pd.DataFrame(kb.scores_)
scores["Variable"]=X.columns
scores.rename(columns={0:"Score"},inplace=True)

In [227]:
scores=scores.sort_values(by="Score",ascending=False)
scores.reset_index(drop=True,inplace=True)

In [228]:
scores

,Score,Variable
0,0.315545,c_GarageArea
1,0.218927,c_OpenPorchSF
2,0.212038,c_2ndFlrSF
3,0.177057,c_BsmtUnfSF
4,0.134504,c_BsmtFinSF1
5,0.112459,c_LotArea
6,0.104803,c_MasVnrArea
7,0.102961,c_WoodDeckSF
8,0.079668,c_HalfBath
9,0.052024,c_BedroomAbvGr


In [229]:
X[list(scores[:20]["Variable"].values)][cols]

,c_BsmtFinSF1,c_BsmtUnfSF,c_2ndFlrSF,c_GarageArea,c_OpenPorchSF
0,384.0,0.0,0.0,572.0,50.0
1,0.0,816.0,0.0,528.0,0.0
2,0.0,936.0,0.0,576.0,32.0
3,637.0,849.0,0.0,480.0,0.0
4,216.0,700.0,826.0,424.0,169.0
...,...,...,...,...,...
759,0.0,691.0,0.0,216.0,20.0
760,0.0,1372.0,0.0,529.0,140.0
761,504.0,319.0,0.0,506.0,0.0
762,1480.0,120.0,0.0,482.0,53.0


## VarClus

- Varclus es un buen algoritmo de reducción de dimensiones. Aquí hay una breve descripción:
- Se elige un grupo para dividir.
- El grupo elegido se divide en dos grupos al encontrar los dos primeros componentes principales, realizar una rotación ortoblicua y asignar cada variable al componente girado con el que tiene la correlación al cuadrado más alto.
- Las variables se reasignan iterativamente a los conglomerados para maximizar la varianza contabilizada por los componentes del conglomerado]

In [230]:
from varclushi import VarClusHi

In [232]:
vc = VarClusHi(X,maxeigval2=1)
#a feature list (feat_list, default all columns of df)
#max second eigenvalue (maxeigval2, default 1)
#max clusters (maxclus, default None)

In [233]:
vc.varclus()

In [234]:
vc.rsquare

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,c_BsmtFinSF1,0.813039,0.070185,0.201073
1,0,c_BsmtUnfSF,0.571135,0.026731,0.440644
2,0,c_BsmtFullBath,0.658772,0.033494,0.353053
3,0,c_WoodDeckSF,0.113338,0.068363,0.951725
4,1,c_2ndFlrSF,0.806199,0.021717,0.198103
5,1,c_HalfBath,0.677375,0.063802,0.344612
6,1,c_BedroomAbvGr,0.461278,0.023546,0.551712
7,1,c_MiscVal,0.016064,0.002602,0.986503
8,2,c_LotArea,0.110562,0.020332,0.907897
9,2,c_MasVnrArea,0.383758,0.028283,0.634178


In [235]:
vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")

,Cluster,Variable,RS_Own,RS_NC,RS_Ratio
0,0,c_BsmtFinSF1,0.813039,0.070185,0.201073
4,1,c_2ndFlrSF,0.806199,0.021717,0.198103
10,2,c_GarageArea,0.591213,0.017786,0.416189


In [236]:
columns=list(vc.rsquare.sort_values(["Cluster","RS_Ratio"],ascending=True).drop_duplicates(subset=["Cluster"],keep="first")["Variable"].values)

In [198]:
columns

['c_BsmtFinSF1', 'c_2ndFlrSF', 'c_GarageArea']

## Arboles de decision

In [237]:
x_train=X_train[[x for x in X_train if x!="tgt_SalePrice"]]
y_train=X_train[[x for x in X_train if x=="tgt_SalePrice"]]

x_test=X_test[[x for x in X_test if x!="tgt_SalePrice"]]
y_test=X_test[[x for x in X_test if x=="tgt_SalePrice"]]

In [238]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [239]:
x_train

,c_LotArea,c_MasVnrArea,v_ExterQual,v_ExterCond,v_BsmtQual,v_BsmtCond,c_BsmtFinSF1,c_BsmtUnfSF,v_HeatingQC,c_2ndFlrSF,v_LowQualFinSF,c_BsmtFullBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,v_Fireplaces,c_GarageArea,v_GarageQual,v_GarageCond,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_MiscVal,v_MSZoning_rl,v_MSZoning_rm,v_Street_pave,v_LotShape_reg,v_LandContour_otro,v_LotConfig_inside,v_LotConfig_otro,v_LandSlope_otro,v_Condition1_otro,v_Condition2_otro,v_BldgType_otro,v_HouseStyle_1story,v_HouseStyle_2story,v_HouseStyle_otro,v_RoofStyle_hip,v_RoofStyle_otro,v_MasVnrType_none,v_MasVnrType_otro,v_Foundation_otro,v_Foundation_pconc,v_BsmtExposure_otro,v_BsmtFinType1_glq,v_BsmtFinType1_otro,v_BsmtFinType1_unf,v_BsmtFinType2_unf,v_Heating_otro,v_CentralAir_y,v_Electrical_otro,v_Electrical_sbrkr,v_Functional_typ,v_GarageType_builtin,v_GarageType_detchd,v_GarageType_otro,v_GarageFinish_rfn,v_GarageFinish_unf,v_PavedDrive_p,v_PavedDrive_y,v_RoofMatl_otro,v_OverallCond_3,v_OverallCond_4,v_OverallCond_5,v_OverallCond_6,v_OverallCond_7,v_OverallCond_8,v_OverallCond_9
0,7134.0,0.0,2,2,2.0,2.0,384.0,0.0,2,0.0,0,0.0,1,0.0,3.0,1,2,1,572.0,2.0,2.0,0.0,50.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,6240.0,0.0,2,2,2.0,2.0,0.0,816.0,2,0.0,360,0.0,1,0.0,3.0,1,2,1,528.0,2.0,2.0,112.0,0.0,0.0,400.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,6000.0,0.0,1,2,1.0,2.0,0.0,936.0,2,0.0,0,0.0,1,0.0,2.0,1,2,0,576.0,2.0,2.0,0.0,32.0,112.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,9937.0,0.0,2,3,2.0,2.0,637.0,849.0,3,0.0,0,1.0,1,0.0,3.0,1,2,0,480.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,17671.0,0.0,3,3,2.0,2.0,216.0,700.0,3,826.0,0,0.0,1,1.0,4.0,1,3,1,424.0,2.0,2.0,0.0,169.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
759,5400.0,0.0,2,2,1.0,2.0,0.0,691.0,3,0.0,0,0.0,1,0.0,2.0,1,3,0,216.0,1.0,2.0,0.0,20.0,94.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
760,8640.0,0.0,3,2,3.0,2.0,0.0,1372.0,3,0.0,0,0.0,2,0.0,3.0,1,3,0,529.0,2.0,2.0,0.0,140.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
761,7763.0,0.0,2,3,2.0,2.0,504.0,319.0,2,0.0,0,1.0,1,0.0,3.0,1,2,0,506.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
762,5587.0,186.0,3,2,3.0,2.0,1480.0,120.0,3,0.0,0,1.0,2,0.0,2.0,1,3,1,482.0,2.0,2.0,162.0,53.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [240]:
y_train

,tgt_SalePrice
0,130000
1,114500
2,93000
3,167000
4,168000
...,...
759,86000
760,250580
761,140000
762,392500


In [241]:
model = DecisionTreeRegressor()

model.fit(x_train, y_train)

importance = model.feature_importances_

In [242]:
model.feature_importances_

array([2.40629194e-02, 3.63384006e-03, 4.45456671e-01, 4.14628500e-05,
       7.13945956e-04, 1.59401517e-04, 8.16477651e-02, 2.03895736e-02,
       3.36404138e-03, 9.70669528e-02, 1.20801468e-04, 1.25019263e-03,
       1.09017924e-02, 3.69832355e-03, 5.24042223e-03, 0.00000000e+00,
       8.67817864e-03, 1.27980950e-02, 1.29998697e-01, 7.71215201e-04,
       9.75383676e-04, 8.78825921e-03, 1.50770074e-02, 3.42170292e-03,
       2.74007033e-04, 1.91074701e-04, 1.36913525e-02, 0.00000000e+00,
       1.68065148e-03, 1.65302430e-02, 8.93206053e-04, 5.10203833e-03,
       1.21514710e-05, 1.31942558e-03, 1.17976302e-06, 6.14926160e-04,
       9.18162444e-04, 4.29981150e-05, 1.84964505e-05, 6.63388638e-04,
       5.66933889e-04, 2.50422745e-03, 2.86339826e-04, 6.47455596e-04,
       8.11187926e-04, 2.05443239e-03, 5.54605522e-04, 4.94745420e-06,
       1.15192061e-04, 1.60868990e-03, 5.91077600e-04, 2.77992825e-02,
       4.55012260e-05, 2.21795378e-04, 8.14822993e-05, 4.48481407e-04,
      

In [202]:
importance

array([2.47293570e-02, 3.21737086e-03, 4.45456671e-01, 1.46873378e-04,
       9.26754940e-04, 3.77820810e-03, 8.17671181e-02, 1.86323062e-02,
       1.92437092e-03, 9.35146375e-02, 2.66743143e-04, 1.71881758e-03,
       1.09082575e-02, 2.34170234e-03, 3.29551764e-03, 3.16141883e-05,
       8.62372605e-03, 1.26530590e-02, 1.30127665e-01, 7.64923132e-04,
       9.91560292e-04, 1.02363416e-02, 1.62215430e-02, 2.82965202e-03,
       2.64032852e-05, 1.25781814e-04, 1.42054973e-02, 0.00000000e+00,
       1.66521713e-03, 1.65445292e-02, 4.43897824e-03, 1.51884043e-03,
       2.70986840e-04, 1.14382750e-03, 1.20807733e-05, 8.30293540e-04,
       8.58023689e-04, 5.07534052e-05, 3.75756666e-05, 8.88967656e-04,
       5.78731519e-04, 2.66744150e-03, 2.97941144e-04, 7.61679473e-04,
       9.93998507e-04, 1.82707272e-03, 8.20927497e-04, 6.52589533e-05,
       5.31994472e-06, 1.62597781e-03, 5.89378741e-04, 2.77992825e-02,
       1.40455982e-04, 9.88764631e-05, 1.52541408e-03, 3.28219511e-04,
      

In [203]:
pred_test=model.predict(x_test)
pred_train=model.predict(x_train)

In [206]:
importance

array([2.47293570e-02, 3.21737086e-03, 4.45456671e-01, 1.46873378e-04,
       9.26754940e-04, 3.77820810e-03, 8.17671181e-02, 1.86323062e-02,
       1.92437092e-03, 9.35146375e-02, 2.66743143e-04, 1.71881758e-03,
       1.09082575e-02, 2.34170234e-03, 3.29551764e-03, 3.16141883e-05,
       8.62372605e-03, 1.26530590e-02, 1.30127665e-01, 7.64923132e-04,
       9.91560292e-04, 1.02363416e-02, 1.62215430e-02, 2.82965202e-03,
       2.64032852e-05, 1.25781814e-04, 1.42054973e-02, 0.00000000e+00,
       1.66521713e-03, 1.65445292e-02, 4.43897824e-03, 1.51884043e-03,
       2.70986840e-04, 1.14382750e-03, 1.20807733e-05, 8.30293540e-04,
       8.58023689e-04, 5.07534052e-05, 3.75756666e-05, 8.88967656e-04,
       5.78731519e-04, 2.66744150e-03, 2.97941144e-04, 7.61679473e-04,
       9.93998507e-04, 1.82707272e-03, 8.20927497e-04, 6.52589533e-05,
       5.31994472e-06, 1.62597781e-03, 5.89378741e-04, 2.77992825e-02,
       1.40455982e-04, 9.88764631e-05, 1.52541408e-03, 3.28219511e-04,
      

In [243]:
scores=pd.DataFrame()
scores["gini_index"]=importance
scores["feature"]=x_train.columns

In [244]:
scores=scores.sort_values(by="gini_index",ascending=False)
scores.reset_index(drop=True,inplace=True)

In [245]:
feats=list(scores[:10]["feature"].values)

In [246]:
scores

,gini_index,feature
0,0.445457,v_ExterQual
1,0.129999,c_GarageArea
2,0.097067,c_2ndFlrSF
3,0.081648,c_BsmtFinSF1
4,0.030149,v_GarageFinish_unf
...,...,...
65,0.000001,v_Condition2_otro
66,0.000000,v_RoofMatl_otro
67,0.000000,v_KitchenAbvGr
68,0.000000,v_GarageType_otro
